In [1]:
%pip install itables tqdm


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [28]:
import math
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os.path
from tqdm import tqdm
from scipy.stats import entropy
from itertools import permutations


def safe_log2(x):
    return math.log2(x) if x > 0 else 0


def tuple_to_string(num_tuple):
    return ''.join(map(str, num_tuple))


def calculate_bulls_cows(source, target):
    if len(source) != len(target):
        raise ValueError("Input arrays must have the same length")

    bulls = sum(s == t for s, t in zip(source, target))
    common_digits = set(source) & set(target)
    cows = sum(min(source.count(digit), target.count(digit)) for digit in common_digits) - bulls

    return bulls, cows


def parse_bulls_n_cows_map_name(digits, guesses={}):
	suffix = ''.join(f'_{k}:{v[0]}{v[1]}' for (k, v) in sorted(guesses.items()))
	return f'bulls_n_cows_map/{digits}{suffix}.pkl'


def initialize(originals, digits):
    filepath = parse_bulls_n_cows_map_name(digits=digits)
    if os.path.isfile(filepath):
        return

    bulls_n_cows_map = {}
    for i, _ in enumerate(originals):
        bulls_n_cows_map[i] = {}
        for di in range(digits+1):
            for dj in range(di+1):
                bulls_n_cows_map[i][(dj, di-dj)] = set()

    for i, source in tqdm(enumerate(originals), total=len(originals)):
        bulls_n_cows_map[i][(digits, 0)].add(i)
        for j in range(i):
            target = originals[j]
            bulls_n_cows = calculate_bulls_cows(source, target)
            bulls_n_cows_map[i][bulls_n_cows].add(j)
            bulls_n_cows_map[j][bulls_n_cows].add(i)

    with open(filepath, 'wb') as f:
    	pickle.dump(bulls_n_cows_map, f, protocol=pickle.HIGHEST_PROTOCOL)


def convert_bulls_n_cows_map(originals, bulls_n_cows_map):
	return {originals[i]: {bc: set(originals[j] for j in bulls_n_cows_map[i][bc]) for bc in bulls_n_cows_map[i] if len(bulls_n_cows_map[i][bc]) > 0} for i in bulls_n_cows_map}


def read_bulls_n_cows_map(digits, curr_guesses={}):
	filepath = parse_bulls_n_cows_map_name(digits, curr_guesses)
	if not os.path.isfile(filepath):
		return None

	with open(filepath, 'rb') as f:
		bulls_n_cows_map = pickle.load(f)

	return bulls_n_cows_map


def update_bulls_n_cows_map(org_idx_map, guess, bulls_n_cows, digits, curr_guesses={}):
    next_guesses = curr_guesses.copy()
    next_guesses[guess] = bulls_n_cows

    filepath = parse_bulls_n_cows_map_name(digits=digits, guesses=next_guesses)
    if os.path.isfile(filepath):
        return read_bulls_n_cows_map(digits=digits, curr_guesses=next_guesses)

    bulls_n_cows_map = read_bulls_n_cows_map(digits=digits, curr_guesses=curr_guesses)

    guess_idx = org_idx_map[guess]	
    candidates = bulls_n_cows_map[guess_idx][bulls_n_cows]

    for src_idx in bulls_n_cows_map:
        for bc in bulls_n_cows_map[src_idx]:
            bulls_n_cows_map[src_idx][bc] = bulls_n_cows_map[src_idx][bc].intersection(candidates)

    with open(filepath, 'wb') as f:
        pickle.dump(bulls_n_cows_map, f, protocol=pickle.HIGHEST_PROTOCOL)
        
    return bulls_n_cows_map


def calc_candidates(bulls_n_cows_map):
    candidates = set()
    for src_idx in bulls_n_cows_map:
        for bc in bulls_n_cows_map[src_idx]:
            candidates = candidates.union(bulls_n_cows_map[src_idx][bc])

    return candidates


def calc_entropy(bulls_n_cows_map):
    entropy_map = {}
    for src_idx in bulls_n_cows_map:
        entropy_map[src_idx] = entropy([len(bulls_n_cows_map[src_idx][bc]) for bc in bulls_n_cows_map[src_idx]], base=2)

    return entropy_map


def try_guess(originals, org_idx_map, digits, guesses, guess, bulls_n_cows, verbose=False):
    if verbose:
        bulls_n_cows_map = read_bulls_n_cows_map(digits=digits, curr_guesses=guesses)
        print('--', convert_bulls_n_cows_map(originals=originals, bulls_n_cows_map=bulls_n_cows_map))

    bulls_n_cows_map = update_bulls_n_cows_map(org_idx_map=org_idx_map, digits=digits, curr_guesses=guesses, guess=guess, bulls_n_cows=bulls_n_cows)
    
    if verbose:
        print('++', convert_bulls_n_cows_map(originals=originals, bulls_n_cows_map=bulls_n_cows_map))

    guesses[guess] = bulls_n_cows
    candidates = [originals[c] for c in calc_candidates(bulls_n_cows_map=bulls_n_cows_map)]
    entropy_map = calc_entropy(bulls_n_cows_map=bulls_n_cows_map)
    best_guess = max(entropy_map, key=entropy_map.get)

    print(guesses, f": {safe_log2(len(candidates)):.2f} bits - {entropy_map[best_guess]:.2f} bits ('{originals[best_guess]}')", candidates)

    return len(candidates), originals[best_guess], entropy_map[best_guess]

In [37]:
class BullsNCows:
    def __init__(self, digits=4):
        permute = permutations([i for i in range(10)], digits)
        self.originals = [tuple_to_string(p) for p in list(permute)]
        self.org_idx_map = {org: idx for idx, org in enumerate(self.originals)}
        self.digits = digits
        self.reset()

        print(self.secret)
        initialize(originals=self.originals, digits=4)


    def reset(self):
        self.guesses = {}
        self.secret = random.choice(self.originals)
        self.summary = [{
            "candidate_count": len(self.originals),
            "candidate_entropy": safe_log2(len(self.originals)),
        }]


    def next(self):
        if len(self.guesses) > 0:
            guess = self.summary[-1]['best_guess']
        else:
            guess = random.choice(self.originals)
 
        guess_result = calculate_bulls_cows(self.secret, guess)
        candidate_count, best_guess, best_guess_entropy = try_guess(originals=self.originals, org_idx_map=self.org_idx_map, guesses=self.guesses, guess=guess, bulls_n_cows=guess_result, digits=self.digits)

        self.summary[-1]['guess'] = guess
        self.summary[-1]['guess_result'] = guess_result
        self.summary[-1]['guess_actual_entropy'] = self.summary[-1]['candidate_entropy']-safe_log2(candidate_count)
        self.summary.append({
            "best_guess": best_guess,
            "best_guess_entropy": best_guess_entropy,
            "candidate_count": candidate_count,
            "candidate_entropy": safe_log2(candidate_count),
        })

        return self


    def play(self, n_iter=10):
        self.reset()
        while self.summary[-1]['candidate_count'] > 1 and n_iter > 0:
            self.next()
            n_iter -= 1

        return pd.DataFrame.from_dict(self.summary)

In [39]:
df = BullsNCows(digits=4).play(); df

1458
{'6891': (0, 2)} : 10.30 bits - 2.94 bits ('0162') ['8123', '8124', '8125', '0126', '8127', '0128', '0129', '8130', '8132', '8134', '0136', '8135', '0138', '0139', '8137', '8140', '8142', '0146', '8143', '0148', '0149', '8145', '8147', '8150', '0156', '8152', '0158', '0159', '0162', '0163', '0164', '0165', '0167', '8157', '8170', '8172', '8173', '8174', '8175', '0176', '0178', '0179', '0182', '0183', '0184', '0185', '9630', '0187', '8206', '9632', '8209', '8210', '8213', '8214', '8215', '0216', '8217', '0218', '0219', '9634', '8236', '8239', '8246', '8249', '9640', '8256', '8259', '8260', '8263', '8264', '8265', '8267', '0268', '0269', '9642', '9643', '8276', '8279', '0286', '0289', '8306', '8309', '8310', '8312', '8314', '8315', '0316', '8317', '0318', '0319', '9624', '9650', '8326', '9652', '8329', '9653', '8346', '8349', '8356', '8359', '8360', '8362', '8364', '8365', '8367', '0368', '0369', '8376', '8379', '0386', '0389', '8406', '8409', '8410', '8412', '8413', '8415', '0416',

,candidate_count,candidate_entropy,guess,guess_result,guess_actual_entropy,best_guess,best_guess_entropy
0,5040,12.299208,6891,"(0, 2)",2.000000,NaN,NaN
1,1260,10.299208,0162,"(0, 2)",2.164782,0162,2.941456
2,281,8.134426,1039,"(2, 0)",3.610864,1039,3.196319
3,23,4.523562,2489,"(3, 0)",4.523562,2489,2.942496
4,1,0.000000,NaN,NaN,NaN,0123,0.000000


In [30]:
game = BullsNCows(digits=4)
pd.DataFrame.from_dict(game.next().next().next().next().next().next().summary)

0428
{'9045': (0, 2)} : 10.30 bits - 2.94 bits ('0124') ['0124', '0129', '0134', '0139', '8150', '0152', '0153', '8154', '0156', '0157', '0158', '8159', '0164', '0169', '0174', '0179', '8190', '0184', '8194', '0189', '0192', '0193', '8204', '0196', '0197', '0198', '8209', '0214', '0219', '0234', '0239', '8250', '0251', '0253', '8254', '0256', '0257', '0258', '8259', '0264', '0269', '0274', '0279', '8290', '0284', '8294', '0289', '0291', '0293', '8304', '0296', '0297', '0298', '8309', '0314', '0319', '0324', '0329', '8350', '0351', '0352', '8354', '0356', '0357', '0358', '8359', '0364', '0369', '0374', '0379', '8390', '0384', '8394', '0389', '0391', '0392', '8401', '8402', '0396', '0397', '0398', '0412', '0413', '8403', '0416', '0417', '0418', '8407', '0421', '0423', '8419', '0426', '0427', '0428', '8420', '0431', '0432', '8429', '0436', '0437', '0438', '8430', '8439', '8451', '8452', '8453', '8456', '8457', '8460', '0461', '0462', '0463', '0467', '0468', '8469', '0471', '0472', '0473',

,candidate_count,candidate_entropy,guess,guess_result,guess_actual_entropy,best_guess,best_guess_entropy
0,5040,12.299208,9045,"(0, 2)",2.000000,NaN,NaN
1,1260,10.299208,0124,"(2, 1)",5.598768,0124,2.941456
2,26,4.700440,0436,"(2, 0)",3.700440,0436,2.997953
3,2,1.000000,0127,"(2, 0)",1.000000,0127,1.000000
4,1,0.000000,0123,"(2, 0)",0.000000,0123,0.000000
5,1,0.000000,0123,"(2, 0)",0.000000,0123,0.000000
6,1,0.000000,NaN,NaN,NaN,0123,0.000000


In [40]:
# guesses={}
# try_guess(originals=game.originals, org_idx_map=game.org_idx_map, guesses=guesses, guess='5142', bulls_n_cows=(1, 0), digits=4)
# try_guess(originals=game.originals, org_idx_map=game.org_idx_map, guesses=guesses, guess='2341', bulls_n_cows=(1, 1), digits=4)
# try_guess(originals=game.originals, org_idx_map=game.org_idx_map, guesses=guesses, guess='3628', bulls_n_cows=(2, 0), digits=4)

guesses={'5142': (1, 0), '2341': (1, 1)}
try_guess(originals=game.originals, org_idx_map=game.org_idx_map, guesses=guesses, guess='0128', bulls_n_cows=(0, 1), digits=4, verbose=False)
try_guess(originals=game.originals, org_idx_map=game.org_idx_map, guesses=guesses, guess='3846', bulls_n_cows=(2, 1), digits=4, verbose=False)

{'5142': (1, 0), '2341': (1, 1), '0128': (0, 1)} : 4.58 bits - 3.16 bits ('3846') ['9362', '3846', '3847', '3849', '9372', '3740', '9043', '3640', '7362', '9843', '6372', '8943', '7043', '7392', '8743', '6043', '6392', '8643', '7843', '6843', '3046', '3047', '3049', '3940']
{'5142': (1, 0), '2341': (1, 1), '0128': (0, 1), '3846': (2, 1)} : 1.58 bits - 1.58 bits ('0167') ['7843', '3640', '9843']


(3, '0167', 1.584962500721156)

In [42]:
game = BullsNCows(digits=2)
initialize(originals=game.originals, digits=2)

guesses={}
try_guess(originals=game.originals, org_idx_map=game.org_idx_map, guesses=guesses, guess='54', bulls_n_cows=(1, 0), digits=2)
try_guess(originals=game.originals, org_idx_map=game.org_idx_map, guesses=guesses, guess='12', bulls_n_cows=(0, 1), digits=2)
try_guess(originals=game.originals, org_idx_map=game.org_idx_map, guesses=guesses, guess='24', bulls_n_cows=(2, 0), digits=2)

83
{'54': (1, 0)} : 4.00 bits - 1.54 bits ('04') ['74', '04', '84', '14', '24', '94', '34', '50', '51', '52', '53', '56', '57', '58', '59', '64']
{'54': (1, 0), '12': (0, 1)} : 1.00 bits - 1.00 bits ('01') ['24', '51']
{'54': (1, 0), '12': (0, 1), '24': (2, 0)} : 0.00 bits - 0.00 bits ('01') ['24']


(1, '01', 0.0)

In [43]:
game = BullsNCows(digits=3)
initialize(originals=game.originals, digits=3)

guesses={}
try_guess(originals=game.originals, org_idx_map=game.org_idx_map, guesses=guesses, guess='514', bulls_n_cows=(1, 0), digits=3)
try_guess(originals=game.originals, org_idx_map=game.org_idx_map, guesses=guesses, guess='138', bulls_n_cows=(1, 1), digits=3)
try_guess(originals=game.originals, org_idx_map=game.org_idx_map, guesses=guesses, guess='239', bulls_n_cows=(1, 0), digits=3)

840
{'514': (1, 0)} : 6.98 bits - 2.33 bits ('012') ['710', '013', '713', '712', '016', '716', '718', '719', '017', '018', '024', '724', '019', '012', '374', '572', '034', '734', '384', '364', '394', '064', '764', '074', '784', '084', '794', '804', '094', '810', '812', '813', '816', '817', '819', '824', '834', '502', '503', '506', '507', '508', '509', '864', '520', '523', '874', '526', '527', '528', '529', '530', '532', '894', '536', '537', '538', '539', '904', '910', '912', '913', '204', '560', '916', '917', '918', '210', '213', '562', '924', '216', '217', '218', '219', '569', '570', '234', '934', '573', '576', '578', '579', '580', '582', '583', '586', '587', '589', '590', '592', '593', '596', '597', '598', '604', '610', '612', '613', '264', '964', '617', '618', '619', '274', '974', '624', '284', '984', '634', '294', '304', '310', '312', '316', '317', '318', '319', '674', '324', '563', '684', '694', '567', '704', '568']
{'514': (1, 0), '138': (1, 1)} : 2.58 bits - 1.79 bits ('012') ['

(2, '012', 1.0)